In [1]:
# load data

import torch
import copy
data_name='few_shot'
file_path = 'data/cora_fixed_sbert.pt'
data_org = torch.load(file_path, map_location='cpu')# data for test

# org data for test
#print(data_org.y[14])

# data for train
data_train = copy.deepcopy(data_org)  
#print(data_train.label_names)
all_number=2708



/tmp/ipykernel_781623/572946539.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_org = torch.load(file_path, map_location='cpu')# data for test
/home/chunjielu/minic

In [2]:
# randomly set train& test dataset on GNN

def random_split_masks(data, num_train=140, num_val=500, num_test=1000, seed=32):

    """
    randomly set the data for train, val, test with certain number
    """
    torch.manual_seed(seed)  

    num_nodes = data.num_nodes
    all_indices = torch.randperm(num_nodes)  

    # index rearrange
    train_idx = all_indices[:num_train]
    val_idx = all_indices[num_train:num_train + num_val]
    test_idx = all_indices[num_train + num_val:num_train + num_val + num_test]


    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    data.train_mask[train_idx] = True
    data.val_mask[val_idx] = True
    data.test_mask[test_idx] = True

    return data 

data_org = random_split_masks(data_org)
data_train = random_split_masks(data_train)

"""
print("Train size:", data_train.train_mask.sum().item())
print("Val size:", data_train.val_mask.sum().item())
print("Test size:", data_train.test_mask.sum().item())
print("是否有交集:", (data_train.train_mask & data_train.val_mask).any().item() or 
                   (data_train.train_mask & data_train.test_mask).any().item() or 
                   (data_train.val_mask & data_train.test_mask).any().item())
"""

'\nprint("Train size:", data_train.train_mask.sum().item())\nprint("Val size:", data_train.val_mask.sum().item())\nprint("Test size:", data_train.test_mask.sum().item())\nprint("是否有交集:", (data_train.train_mask & data_train.val_mask).any().item() or \n                   (data_train.train_mask & data_train.test_mask).any().item() or \n                   (data_train.val_mask & data_train.test_mask).any().item())\n'

In [4]:
import torch

# load data
file_path = 'LLM_GNN_data/cora_openai.pt'  
data = torch.load(file_path, map_location='cpu')


consistency_data = data[data_name]
class_map = { # a mapping from the responses
    'rule_Learning': 0,
    'neural_networks': 1,
    "case_based": 2,
    'genetic_algorithms': 3,
    'theory': 4,
    "reinforcement_learning": 5,
    "probabilistic_methods": 6
}
#print(consistency_data)


/tmp/ipykernel_781623/2198844873.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path, map_location='cpu')


In [5]:
import networkx as nx
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import numpy as np
from sklearn.cluster import KMeans
from torch_geometric.datasets import Planetoid
from sklearn.metrics.pairwise import euclidean_distances
import faiss

def get_rank_score(rank_list,n,m):
    result=[0 for i in range(n)]
    for i in range(m):
        k=rank_list[i]
        result[k]=100*(m-i)/m
    return result
def compute_rC_density(node_features: torch.Tensor, cluster_centers: torch.Tensor, labels: torch.Tensor):
    """
    cal the C-density score

        node_features: [N, D] node feature
        cluster_centers: [K, D] cluster centure
        labels: [N] the label

    """
    assigned_centers = cluster_centers[labels]        
    distances = torch.norm(node_features - assigned_centers, dim=1)  
    c_density = 1 / (1 + distances)  
    return c_density


def compute_cluster_rank(data, num_clusters=7):
    """
    implement k-means then get c_density score , then return the ranking
    """

    x = data.x.detach().cpu().float()
    n, d = x.shape

    kmeans = faiss.Kmeans(d=d, k=num_clusters, niter=20, verbose=False, seed=42)
    kmeans.train(x.numpy())


    _, I = kmeans.index.search(x.numpy(), 1)  # I: [N, 1]
    labels = torch.tensor(I.squeeze(), dtype=torch.long)  # [N]

    centers = torch.tensor(kmeans.centroids, dtype=x.dtype)  # [K, D]
    c_density = compute_rC_density(x, centers, labels)  # [N]
    sorted_idx = torch.argsort(c_density, descending=True)  # [N]

    rank_list = sorted_idx.tolist()
    return get_rank_score(rank_list, data.num_nodes,data.num_nodes)


cluster_list=compute_cluster_rank(data_train)
#print(cluster_list)
# get R_c-density for every node

In [6]:

# process the responses (string)
unknown_labels = set()

def get_result(s):
    """
    return the class [0,7) and it's confidence score from the string returned by LLM
    """
    l=0
    while(s[l]!=":"):
        l+=1
    l+=2
    if s[l-1]!='"':
        l+=1

    r=l
    while(s[r]!='"'):
        r+=1
    label_str=s[l:r]
    #print(label_str)
    conf=0
    for i in s:
        if ord(i)>=ord('0') and ord(i)<=ord('9'):
            conf=conf*10+ord(i)-ord('0')

    if label_str not in class_map:
        unknown_labels.add(label_str)
    
    else:
        return class_map[label_str],conf

    


# result store the value we need (index,(prediction, confidence score))
result=[]
for i, item in enumerate(consistency_data):
    if not item or not isinstance(item, list) or not any(s.strip() for s in item):
        continue
    if get_result(item[0])!=None:
        result.append([i,get_result(item[0])])
    #print(i,get_result(item[0]))

#print(result)

total=0
op=0
mask_list=[] # mask list to give a train mask
label_list=[]# a list of label with order of mask_list
anao=[-1 for i in range(len(consistency_data))]
# all 2078 annotations, -1 means it has on response form openai 
conf_list=[]
for i in range(len(result)):
    idx=result[i][0]
    anao[idx]=result[i][1][1]/100
    conf_list.append(result[i][1][1]/100)
    label=result[i][1][0]
    mask_list.append(idx)
    label_list.append(label)
    if data_org.y[idx].item()==label:
        op+=1
    total+=1
#print(op/total,total)
print(f'annotation accuracy {op/total} with {total} nodes')
# turn into tensor
mask_list_tensor=torch.tensor(mask_list)
label_list_tensor=torch.tensor(label_list)
annotations=torch.tensor(anao)


#print(mask_list)


annotation accuracy 0.7097966728280961 with 541 nodes


In [7]:
import numpy as np
import torch
from collections import Counter
from scipy.stats import entropy
def compute_entropy_from_labels(labels):
    """
    cal  Shannon entropy
    labels: List[int] or 1D tensor
    """
    labels = labels.cpu().numpy() if isinstance(labels, torch.Tensor) else np.array(labels)
    counter = Counter(labels)
    probs = np.array([v / len(labels) for v in counter.values()])
    return entropy(probs, base=np.e)  # Shannon entropy in natural log base


def compute_rCOE(selected_nodes, annotations, return_rank=True):
    """
    cal every node's COE score return ranking
    """
    if isinstance(selected_nodes, torch.Tensor):
        selected_nodes = selected_nodes.tolist()

    selected_labels = annotations[selected_nodes]
    base_entropy = compute_entropy_from_labels(selected_labels)

    coe_scores = {}
    for node in selected_nodes:
        # subset apart from one node
        remaining_nodes = [n for n in selected_nodes if n != node]
        remaining_labels = annotations[remaining_nodes]
        new_entropy = compute_entropy_from_labels(remaining_labels)
        coe_scores[node] = new_entropy - base_entropy

    if return_rank:
        sorted_nodes = sorted(coe_scores.items(), key=lambda x: x[1], reverse=True)
        ranked_nodes = [node for node, _ in sorted_nodes]
        return get_rank_score(ranked_nodes,all_number,len(ranked_nodes))

    return coe_scores

def get_rank_score(rank_list,n,m):
    result=[-1 for i in range(n)]
    for i in range(m):
        k=rank_list[i]
        result[k]=100*(m-i)/m
    return result

def get_confidence_rank_score(selected_nodes,conf):
    result=[]
    for i in range(len(selected_nodes)):
        result.append((selected_nodes[i],conf[i]))

    sorted_result=sorted(result, key=lambda x: x[1], reverse=True)
    rank_list = [node for node, _ in sorted_result]
    #print(len(rank_list))
    score_list=get_rank_score(rank_list,all_number,len(rank_list))

    return score_list
"""    
# conf rank and coe rank
conf_rank=get_confidence_rank_score(mask_list,conf_list)
print(conf_rank)
rank=compute_rCOE(mask_list,annotations)
print(len(rank))

print(conf_rank)
for i in conf_rank:
    print(anao[i])
"""



    



'    \n# conf rank and coe rank\nconf_rank=get_confidence_rank_score(mask_list,conf_list)\nprint(conf_rank)\nrank=compute_rCOE(mask_list,annotations)\nprint(len(rank))\n\nprint(conf_rank)\nfor i in conf_rank:\n    print(anao[i])\n'

In [8]:

def get_filter_out_index(mask_list,label_list,conf_list,annotations,cluster_list):
    """
    get score= c-density-score+cond_score+COE_score
    """
    conf_score=get_confidence_rank_score(mask_list,conf_list)
    COE_score=compute_rCOE(mask_list,annotations)

    op=-1
    val=-1
    for i in range(len(COE_score)):
        if conf_score[i]==-1:
            continue
        score=0.25*conf_score[i]+0.5*COE_score[i]+0.25*cluster_list[i]
        if op==-1 or score<val:
            val=score
            op=i
    return op


def post_filter(final_number,mask_list,label_list,conf_list,annotations,cluster_list):
    """
    return the list with the certain number(final_number) [mask_list,label_list]
    using score= c-density-score+cond_score+COE_score

    """
    process_num=0
    if final_number>=len(mask_list):
        print('No nodes will be filtered out')
        return [mask_list,label_list]
    for k in range(len(mask_list)-final_number):
        process_num+=1
        j=get_filter_out_index(mask_list,label_list,conf_list,annotations,cluster_list)
        if process_num%40==0:
            print(f'process {process_num} nodes')
        #print(j)
        new_label_list=[]
        new_conf_list=[]
        new_mask_list=[]
        for i in range(len(mask_list)):
            if mask_list[i]==j:
                #print(mask_list[i],j)
                continue
            new_label_list.append(label_list[i])
            new_conf_list.append(conf_list[i])
            new_mask_list.append(mask_list[i])
        
        mask_list = new_mask_list
        label_list = new_label_list
        conf_list = new_conf_list
    
    return [mask_list,label_list]


A,B=post_filter(300,mask_list,label_list,conf_list,annotations,cluster_list)

            
print(len(A))
#print(len(B))



        
    

process 40 nodes
process 80 nodes
process 120 nodes
process 160 nodes
process 200 nodes
process 240 nodes
300


In [9]:
def change_trainmask_and_label(mask_list,label_list,data):
    train_mask = torch.zeros(data_train.num_nodes, dtype=torch.bool)
    train_mask[mask_list] = True
    data.train_mask = train_mask
    data.y[data_train.train_mask] = label_list


change_trainmask_and_label(A,torch.tensor(B),data_train)

train_nodes = data_train.train_mask.sum().item()  
print(f'Number of training nodes: {train_nodes}')



Number of training nodes: 300


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class GCN2(torch.nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dimension, num_classes, dropout, norm=None) -> None:
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = dropout
        if num_layers == 1:
            self.convs.append(GCNConv(input_dim, num_classes, cached=False,
                             normalize=True))
        else:
            self.convs.append(GCNConv(input_dim, hidden_dimension, cached=False,
                             normalize=True))
            if norm:
                self.norms.append(torch.nn.BatchNorm1d(hidden_dimension))
            else:
                self.norms.append(torch.nn.Identity())

            for _ in range(num_layers - 2):
                self.convs.append(GCNConv(hidden_dimension, hidden_dimension, cached=False,
                             normalize=True))
                if norm:
                    self.norms.append(torch.nn.BatchNorm1d(hidden_dimension))
                else:
                    self.norms.append(torch.nn.Identity())

            self.convs.append(GCNConv(hidden_dimension, num_classes, cached=False, normalize=True))

    def forward(self, data):
        x, edge_index, edge_weight= data.x, data.edge_index, data.edge_weight
        for i in range(self.num_layers):
            x = F.dropout(x, p=self.dropout, training=self.training)
            if edge_weight != None:
                x = self.convs[i](x, edge_index, edge_weight)
            else:
                x = self.convs[i](x, edge_index)
            if i != self.num_layers - 1:
                x = self.norms[i](x)
                x = F.relu(x)
        return x

model = GCN2(num_layers=2, input_dim=data_train.num_node_features, hidden_dimension=64, 
            num_classes=7, dropout=0.5, norm=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# 训练函数
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs  # train_acc, val_acc, test_acc
def test_final(data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs  # train_acc, val_acc, test_acc

# 训练流程
for epoch in range(1, 80):
    loss = train(data_train)
    train_acc, val_acc = test(data_train)
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}")

Epoch 001, Loss: 2.3452, Train: 0.1433, Val: 0.1440
Epoch 020, Loss: 0.6369, Train: 0.8367, Val: 0.6840
Epoch 040, Loss: 0.4575, Train: 0.8800, Val: 0.6940
Epoch 060, Loss: 0.3784, Train: 0.9067, Val: 0.7160


In [11]:
print(f'final accuracy: {test_final(data_org)}')

final accuracy: [0.742]
